In [2]:
import numpy as np
import pandas as pd
# Use another model to get new features

In [3]:
import csv
import numpy as np

cnt = 1
csv_file_path = 'crop_production.csv'
states = {}
districts = {}
season_to_number = {}
crops = {}
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in  csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        states[row[0].strip()] = 1
        districts[row[1].strip()] = 1
        season_to_number[row[3].strip()] = 1
        crops[row[4].strip()] = 1
        
cnt = 1
for val in states.keys():
    states[val] = cnt
    cnt = cnt+1
cnt = 1
for val in districts.keys():
    districts[val] = cnt
    cnt = cnt+1
cnt = 1

for val in season_to_number.keys():
    season_to_number[val] = cnt
    cnt = cnt+1

len_of_months = {}
len_of_months['Kharif'] = 5
len_of_months['Autumn'] =3
len_of_months['Rabi'] = 6
len_of_months['Summer'] = 3
len_of_months['Winter'] = 3
len_of_months['Whole Year'] = 12
print(season_to_number)
cnt = 1
for val in crops.keys():
    crops[val] = cnt
    cnt = cnt+1



# New idea is to divide the crop produce by area so that we can predict for each district what yield would it give per area then
# multiply by area.

{'Season': 1, 'Kharif': 2, 'Whole Year': 3, 'Autumn': 4, 'Rabi': 5, 'Summer': 6, 'Winter': 7}


In [4]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split


csv_file_path = 'crop_production.csv'

y = []
data = []
cnt = 0
with open(csv_file_path, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        cnt = cnt+1
        if cnt == 1 : continue
        try:
            val1 = float(row[6].strip())
            val2 = float(row[5])
            y.append(val1/val2)
        except Exception as e:
            # print(e)
            continue
        temp_row = row
        month = temp_row[3].strip()
        temp_row[1] = districts[temp_row[1].strip()]
        temp_row[3] = season_to_number[temp_row[3].strip()]
        temp_row[4] = crops[temp_row[4].strip()]
        
        temp1 = row[1:2]
        temp2 = row[3:5]
        # temp3 = [len_of_months[month]]
        
        data.append(temp1+temp2)
       
y = np.array(y)

data = np.array(data)
print(data.shape)
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(data)
# i = 7
# print(data[i])
# print(y[i])


X_train, X_test, y_train, y_test = train_test_split(X_train_normalized, y, test_size = .15, random_state = 18)
X_train , X_cv , y_train , y_cv = train_test_split(X_train_normalized, y, test_size = .10, random_state = 18)

(242361, 3)


In [5]:
print(X_train)
# n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8 

[[-1.36919046  1.0289498   1.98935297]
 [-1.13512263 -0.97610996  1.4156085 ]
 [-1.60834672 -0.30775671 -0.97499346]
 ...
 [-0.36168109 -0.97610996 -0.49687306]
 [-0.34132737 -0.97610996 -1.26186569]
 [ 1.04781434  1.0289498  -0.59249714]]


In [6]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

param_grid_gb = {
    "n_estimators" : [1000],
    "eta" : [1e-03],
    "min_child_weight" : [100,200,300,400],
    "max_depth" : [10,20,30,40,50],
}

model = XGBRegressor()
mse_grid = GridSearchCV(estimator=model, param_grid=param_grid_gb, scoring='neg_mean_squared_error', cv=5, verbose=1)

mse_grid.fit(X_train, y_train, eval_set=[(X_cv, y_cv)], early_stopping_rounds=20,verbose = False)

print("Best parameters:", mse_grid.best_params_)


# Procedure: First tune min_child_weight and max_depth
# Tune gamma
# Tune subsample and colsample
# Tune regularization (alpha paramater)
# Tune the estimators as required

# Previous run (27m 35.4s) Best parameter:  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1}
# Best parameter:  {'colsample_bytree': 1e-05, 'gamma': 0, 'learning_rate': 1e-07, 'max_depth': 1, 'min_child_weight': 1, 'n_estimators': 100, 'reg_alpha': 1e-07, 'subsample': 1}


Fitting 5 folds for each of 20 candidates, totalling 100 fits


c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


KeyboardInterrupt: 

In [15]:
# Best parameter:  {'colsample_bytree': 1, 'gamma': 1e-05, 'learning_rate': 1e-07, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 100, 'reg_alpha': 1e-08, 'subsample': 0.9}


model = XGBRegressor(n_estimators=1000,eta = 1e-03)
# eval_set = [(X_test,y_test)]
model.fit   (X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds = 10)

# print("Best parameter: ", mse_grid.best_params_)

[0]	validation_0-rmse:783.23843	validation_1-rmse:944.15337
[1]	validation_0-rmse:782.92581	validation_1-rmse:943.91505
[2]	validation_0-rmse:782.61365	validation_1-rmse:943.67717


c:\Python311\Lib\site-packages\xgboost\sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-rmse:782.30197	validation_1-rmse:943.43972
[4]	validation_0-rmse:781.99077	validation_1-rmse:943.20272
[5]	validation_0-rmse:781.68004	validation_1-rmse:942.96619
[6]	validation_0-rmse:781.36981	validation_1-rmse:942.73006
[7]	validation_0-rmse:781.06003	validation_1-rmse:942.49434
[8]	validation_0-rmse:780.75072	validation_1-rmse:942.25910
[9]	validation_0-rmse:780.44189	validation_1-rmse:942.02430
[10]	validation_0-rmse:780.13355	validation_1-rmse:941.78993
[11]	validation_0-rmse:779.82568	validation_1-rmse:941.55598
[12]	validation_0-rmse:779.51825	validation_1-rmse:941.32249
[13]	validation_0-rmse:779.21132	validation_1-rmse:941.08941
[14]	validation_0-rmse:778.90484	validation_1-rmse:940.85674
[15]	validation_0-rmse:778.59885	validation_1-rmse:940.62458
[16]	validation_0-rmse:778.29332	validation_1-rmse:940.39278
[17]	validation_0-rmse:777.98824	validation_1-rmse:940.16142
[18]	validation_0-rmse:777.68365	validation_1-rmse:939.93052
[19]	validation_0-rmse:777.3795

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=0.001, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [18]:
# training_error = 0
from sklearn.metrics import mean_squared_error
print("Training error:")
print(mean_squared_error(y_train,model.predict(X_train)))
print("Test error:")
print(mean_squared_error(y_test,model.predict(X_test)))
# Previously
# Training error:
# 2041076666.4211206
# Test error:
# 71631908865818.11
# Overfit
# Now
# Training error:
# 410632.44332906447
# Test error:
# 681743.0108548584
# Ok

Training error:
393392.70268175466
Test error:
712041.6812376654


In [23]:
# This will be used for user inputs
# State_Name = states['Andaman and Nicobar Islands']
s = 'Kharif'
District_Name = districts['NICOBARS']
# Crop_Year = 2000
Season = season_to_number[s]
Crop = crops['Arecanut']
Area = 1254.00
Len = len_of_months[s]
# print(State_Name,District_Name,Crop_Year,Season,Crop,Area)
X_predict = np.zeros((1,3))
X_predict[0] = np.array([District_Name,Season,Crop])
X_pred_normalized = scaler.transform(X_predict)

Production = model.predict(X_pred_normalized)
print(X_pred_normalized)
print(Production*Area)

[[-1.65923103 -0.97610996 -1.30967773]]
[1451.1882]


In [11]:
import pickle
filename = "finalized_model.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [84]:
import pickle
filename = "finalized_model2.sav"
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
# model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [9]:
import pickle
filename = "finalized_model3.sav"
# pickle.dump(model, open(filename, 'wb'))
# model = XGBRegressor(n_estimators=1000,eta = 1e-03)

# some time later...
#  Does not perform well on small aread like 1 10 100
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [11]:
import pickle
filename = "finalized_model4.sav"
# pickle.dump(model, open(filename, 'wb'))
 
# some time later...
#  Does not perform well on small area like 1 10 100
# load the model from disk
model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

In [17]:
import pickle
filename = "finalized_model5.sav"
pickle.dump(model, open(filename, 'wb'))

# model = pickle.load(open(filename, 'rb'))